In [ ]:
import numpy as np
import pandas as pd

########################SET folder to project directory path#####################
folder= ""

import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [ ]:
prolific_export = pd.read_excel(folder+"data\\collected_data\\prolific_export_demographics.xlsx")
## GET data of only approved participants
prolific_export = prolific_export[prolific_export['ACTION']=='Approved']

## GET prolific id of approved participants
participants = prolific_export['Participant id'].tolist()
len(participants)

# Descriptive stats for the participants

## Age

In [ ]:
part_with_age = prolific_export[prolific_export["Age"]!='DATA_EXPIRED']
part_with_age["Age"] = part_with_age["Age"].astype(int)
part_with_age.reset_index(drop = True, inplace = True)

participant_age= part_with_age.groupby(by = ["Participant id","Age"])

participant_age_count = participant_age.size()

data = []

for i in range(0,len(participant_age_count.index)):
    
    data.append(
    {
        "Participant" : participant_age_count.index[i][0],
        "Age" : participant_age_count.index[i][1],
        "Count": participant_age_count.values[i]
    }
    )
participant_age_df = pd.DataFrame(data)
participant_age_df["Age"] = participant_age_df["Age"].astype(int)

participant_age_df['Age'].loc[participant_age_df['Age'] <= 25] = 1
participant_age_df['Age'].loc[(participant_age_df['Age'] >25) & (participant_age_df['Age'] <=40)] = 2
participant_age_df['Age'].loc[(participant_age_df['Age'] >40) & (participant_age_df['Age'] <= 60)] = 3
participant_age_df['Age'].loc[(participant_age_df['Age'] >60)] = 4


participant_age_df.replace({'Age' : { 1 :'Young Adults(<25)', 2 : 'Adults(26-40)',3:"Middle-Aged Adults(41-60)",4:"Older Adults(60+)"}},inplace=True)
participant_age_df.head()

In [ ]:
data = []

for age in participant_age_df["Age"].unique():
    
    data.append(
    {
        "Age Group" : age,
        "Total_Count" : round(participant_age_df[participant_age_df["Age"] == age].Count.sum(),2),
#         "Mean": round(participant_age_df[participant_age_df["Age"] == age].Count.mean(),2),
#         "Std": round(participant_age_df[participant_age_df["Age"] == age].Count.std(),2)
    }
    )    
    
each_participant_age_df = pd.DataFrame(data)

each_participant_age_df.sort_values(by='Total_Count', ascending=False, inplace = True)
each_participant_age_df.fillna("NA", inplace = True)
each_participant_age_df.reset_index(drop = True, inplace = True)

display(each_participant_age_df)

all_ages = list(part_with_age["Age"].value_counts().index)

print("Mean age of all participants: ", np.mean(all_ages))
print("Std of age of all participants: ", round(np.std(all_ages),2))
# each_participant_age_df.to_csv("Each_participant_age.csv")

## Other Columns

In [ ]:
prolific_export.columns

In [ ]:
for column in part_with_age.columns[12:] :
    display(column)    
    current_df = prolific_export[column].value_counts().rename_axis('Groups').reset_index(name='counts')
    display(current_df)